In [1]:
import psutil

# Get total, used, and available memory in GB
mem = psutil.virtual_memory()
print(f"Total RAM: {mem.total / 1e9:.2f} GB")
print(f"Available RAM: {mem.available / 1e9:.2f} GB")
print(f"Used RAM: {mem.used / 1e9:.2f} GB")
print(f"RAM Usage: {mem.percent}%")

!nvidia-smi

Total RAM: 201.34 GB
Available RAM: 180.96 GB
Used RAM: 18.04 GB
RAM Usage: 10.1%
Tue Jul 22 11:50:26 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:3B:00.0 Off |                    0 |
|  0%   27C    P8    12W / 300W |      0MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1 

In [2]:
import torch

# 检查CUDA是否可用
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f"Find {gpu_count} GPU can be used.")

    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu_name}")
else:
    print("No GPU can be used.")


Find 3 GPU can be used.
GPU 1: NVIDIA A40
GPU 2: NVIDIA A40
GPU 3: NVIDIA A40


In [3]:
import copy
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
try:
    from rouge import Rouge
except:
    !pip install rouge
    from rouge import Rouge
# from bert_score import score
import statistics
from ast import literal_eval
import functools
import json
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

random.seed(8888)
torch.manual_seed(8888)
random.seed(8888)
np.random.seed(8888)

if torch.cuda.is_available():
    torch.cuda.manual_seed(8888)
    torch.cuda.manual_seed_all(8888)


from tqdm import tqdm

torch.set_grad_enabled(False)
tqdm.pandas()

In [4]:
import sys
sys.version

'3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]'

In [5]:
torch.cuda.set_device(0)

from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join

model_dir = "../models"
# model_name = 'Meta-Llama-3-2B'  #gemma-2-9b-it  gemma-2-9b
model_name = 'Llama-3.2-1B'
output_dir = '../outputs'
dataset_dir = '../dataset'

#'Meta-Llama-3-8B-Instruct' #'Llama-2-7b-chat-hf' #'llama-7b' 
# "OLMo-7B-Instruct" olmo-2-1124-7B-Instruct 'OLMo-2-1124-7B'  "OLMo-7B-0724-Instruct-hf" olmo1的情况都非常反常，不知道为什么
# Qwen-7B, Qwen1.5-7B, Qwen-7B-Chat, Qwen1.5-7B-Chat, Qwen2-7B-Instruct, Qwen2.5-7B-Instruct, Qwen2.5-Coder-7B, Qwen2.5对中文做过优化，情况不太一样
# Mistral-7B-Instruct-v0.1, Mistral-7B-Instruct-v0.2, Mistral-7B-Instruct-v0.3, Mistral-7B-v0.3
# Yi-1.5-6B-Chat, Yi-6B-Chat
# gemma-2-9b-it gemma-7b-it, gemma-2-9b
# gpt-j-6b, mpt-7b-chat, opt-6.7b, pythia-6.9b, zephyr-7b-beta, falcon-7b-instruct
# deepseek


model = AutoModelForCausalLM.from_pretrained(
    join(model_dir, model_name),
    torch_dtype=torch.float32,
    trust_remote_code=True,
    local_files_only=True
)

tokenizer = AutoTokenizer.from_pretrained(join(model_dir, model_name), trust_remote_code=True)

if 'llama' in model.config.model_type.lower() or 'mistral' in model.config.model_type.lower() or 'yi' in model.config.model_type.lower() or 'gptj' in model.config.model_type.lower():
    tokenizer.pad_token_id = tokenizer.eos_token_id
elif 'qwen' in model.config.model_type.lower():
    tokenizer.pad_token = '<|endoftext|>'
    # in gemma, pad_token_id = 0 is default
    # in olmo, pad_token_id = 1 is default
    
tokenizer.padding_side = "left"

model.to('cuda');

/opt/conda/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
print('model: ',model)
print('model.config: ',model.config)
print('model.config.model_type.lower(): ',model.config.model_type.lower())  # Often provides a string identifier
# print('model.config.num_hidden_layers: ',model.config.num_hidden_layers)


if 'gptj' in model.config.model_type.lower():
    model_layers_num = int(model.config.n_layer)  
    mlp_vector_num = 16384
    mlp_dim_num = int(model.config.n_embd)
    layer_name = 'transformer.h'
    mlp_name = 'mlp'
    mlp_last_layer_name = 'fc_out'

elif 'qwen' in model.config.model_type.lower() and 'qwen2' not in model.config.model_type.lower(): #qwen1, qwen2.5
    layer_name = 'transformer.h'
    mlp_name = 'mlp'
    mlp_last_layer_name = 'w2'
    mlp_dim_num = int(model.config.hidden_size)
    model_layers_num = int(model.config.num_hidden_layers)
    mlp_vector_num = int(model.config.intermediate_size / 2)
    
else:
    model_layers_num = int(model.config.num_hidden_layers)  # on olmo1, olmo2, qwen2, qwen2.5, llama, ...
    mlp_vector_num = int(model.config.intermediate_size)
    mlp_dim_num = int(model.config.hidden_size)
    layer_name = 'model.layers' 
    mlp_name = 'mlp'
    mlp_last_layer_name = 'down_proj'
    attn_name = 'self_attn'

model:  LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rota

In [7]:
import datasets
import json
import re
import random
from datasets import load_from_disk
from tqdm import tqdm

n_new_tokens = 100
NUll_num = 0

def form_options(options: list):
    option_str = 'Options are:\n'
    opts = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
    for opt, o in zip(options, opts):
        option_str += f'({o}): {opt}' + '\n'
    return option_str


def get_prediction(output):
    pattern = r"answer is \(?([ABCDEFGHIJ])\)?"
    match = re.search(pattern, output)
    if match:
        #print('prediction success: ',match.group(1))
        return match.group(1)
    else:
        #print("extraction failed, do a random guess")
        global NUll_num  
        NUll_num += 1
        return random.choice(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])


def generate_outputs(questions):
    
    inputs = tokenizer(questions, return_tensors="pt", padding="longest", return_token_type_ids=False).to('cuda')
    input_length = inputs.input_ids.size(1)
    output = model(**inputs, output_hidden_states = True)
#     Question_input = [[{"role": "user", "content": prompt}] for prompt in questions]
#     texts = tokenizer.apply_chat_template(Question_input ,tokenize=False)

#     inputs = tokenizer(texts, padding="longest", return_tensors="pt")
#     inputs = {key: val.cuda() for key, val in inputs.items()}
#     output = model(**inputs, output_hidden_states = True)
    
    return output

def generate_questions(questions):
    
    inputs = tokenizer(questions, return_tensors="pt", padding="longest", return_token_type_ids=False).to('cuda')
    input_length = inputs.input_ids.size(1)
    gen_tokens = model.generate(**inputs, max_new_tokens=n_new_tokens, do_sample=False)

    gen_text = tokenizer.batch_decode(gen_tokens[:, input_length:], skip_special_tokens=True)
    
    return gen_text


dataset = load_from_disk('../dataset/mmlu-pro')

categories = ['computer science', 'math', 'chemistry', 'engineering', 'law', 'biology',
              'health', 'physics', 'business', 'philosophy', 'economics', 'other',
              'psychology', 'history']

per_category_accuracy = {c: [0, 0] for c in categories}
success, fail = 0, 0
answers = []

print('----------------- Start Answering -------------------')
queries_batch = []  # 可以测试一下batch or single哪种方式准确率更高，更合适一些 #发现基本是一样的，padding不会对准确率造成影响
entry_batch = []

batch_size = 4

random.seed(8888)

test_data = list(dataset['test'])

# example: running on 600 samples
sampled_data = random.sample(test_data, 600)

# layers_to_cache = list(range(model_layers_num))
# print('layers_to_cache: ',layers_to_cache)
# hs_cache_cot = {}
# hs_cache_no_cot = {}

# print('----------------- Running no cot Inference -------------------')
# for ix, entry in tqdm(enumerate(sampled_data)):
        
#     query = 'Q: ' + entry['question'] + "\nA: "
    
#     queries_batch.append(query)
    
#     if len(queries_batch) == batch_size or ix == len(dataset['test']) - 1:
#         output = generate_outputs(queries_batch)
        
#         for layer in layers_to_cache:
#             if layer not in hs_cache_no_cot:
#                 hs_cache_no_cot[layer] = output["hidden_states"][layer][: ,-1 , :].cpu() #bs * tok * dims
#             else:
#                 hs_cache_no_cot[layer] = torch.cat((hs_cache_no_cot[layer], output["hidden_states"][layer][: ,-1 , :].cpu()), dim=0)

        
#         queries_batch = []
#     torch.cuda.empty_cache()
# # TODO print('----------------- Running cot Inference -------------------')

sampled_data = random.sample(test_data, 3000)

layers_to_cache = list(range(model_layers_num))
print('layers_to_cache: ',layers_to_cache)
# hs_cache_cot = {}
hs_cache_no_cot_3000 = {}

print('----------------- Running no cot Inference (3000 samples MMLU-pro) -------------------')
for ix, entry in tqdm(enumerate(sampled_data)):
        
    query = 'Q: ' + entry['question'] + "\nA: "
    
    queries_batch.append(query)
    
    if len(queries_batch) == batch_size or ix == len(dataset['test']) - 1:
        output = generate_outputs(queries_batch)
        
        for layer in layers_to_cache:
            if layer not in hs_cache_no_cot_3000:
                hs_cache_no_cot_3000[layer] = output["hidden_states"][layer][: ,-1 , :].cpu() #bs * tok * dims
            else:
                hs_cache_no_cot_3000[layer] = torch.cat((hs_cache_no_cot_3000[layer], output["hidden_states"][layer][: ,-1 , :].cpu()), dim=0)

        
        queries_batch = []
    torch.cuda.empty_cache()


----------------- Start Answering -------------------
layers_to_cache:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
----------------- Running no cot Inference (3000 samples MMLU-pro) -------------------


3000it [01:21, 36.66it/s]


# **PCA**

In [8]:
with open('../dataset/mmlu-pro_600samples.json', 'r', encoding='utf-8') as f:
    sampled_data = json.load(f)

reason_indices = [ix for ix, sample in enumerate(sampled_data) if sample['memory_reason_score'] > 0.5]
memory_indices = [ix for ix, sample in enumerate(sampled_data) if sample['memory_reason_score'] <= 0.5]

In [9]:
import os
import json
import pandas as pd

In [35]:
# loading and running gsm8k or other dataset


gsm8k_ds_main = load_from_disk('../dataset/gsm8k/main') 
gsm8k_ds_main_test = list(gsm8k_ds_main['test'])

# mbpp_ds_full = load_from_disk('../dataset/mbpp/full')
# mbpp_ds_full_val = list(mbpp_ds_full['validation'])

# mbpp_ds_full = load_from_disk('../dataset/mbpp/full')
# mbpp_ds_full_test = list(mbpp_ds_full['test'])

# example on MGSM, feel free to add other categories in MGSM
mgsm_zh = pd.read_csv('../dataset/mgsm/mgsm_zh.tsv', sep='\t', header=None)
mgsm_zh_test = mgsm_zh.values.tolist()
mgsm_de = pd.read_csv('../dataset/mgsm/mgsm_de.tsv', sep='\t', header=None)
mgsm_de_test = mgsm_de.values.tolist()
mgsm_bn = pd.read_csv('../dataset/mgsm/mgsm_bn.tsv', sep='\t', header=None)
mgsm_bn_test = mgsm_bn.values.tolist()
mgsm_ja = pd.read_csv('../dataset/mgsm/mgsm_ja.tsv', sep='\t', header=None)
mgsm_ja_test = mgsm_ja.values.tolist()
mgsm_te = pd.read_csv('../dataset/mgsm/mgsm_te.tsv', sep='\t', header=None)
mgsm_te_test = mgsm_te.values.tolist()


# example on C-Eval, feel free to add other categories in C-Eval
ceval_chi = pd.read_csv('../dataset/ceval-exam/test/chinese_language_and_literature_test.csv')['question']
ceval_chi_test = ceval_chi.tolist()
ceval_his = pd.read_csv('../dataset/ceval-exam/test/high_school_history_test.csv')['question']
ceval_his_test = ceval_his.tolist()
ceval_pol = pd.read_csv('../dataset/ceval-exam/test/high_school_politics_test.csv')['question']
ceval_pol_test = ceval_pol.tolist()
ceval_mar = pd.read_csv('../dataset/ceval-exam/test/marxism_test.csv')['question']
ceval_mar_test = ceval_mar.tolist()
ceval_bus = pd.read_csv('../dataset/ceval-exam/test/business_administration_test.csv')['question']
ceval_bus_test = ceval_bus.tolist()

popqa_test = load_from_disk('../dataset/PopQA/test') 
popqa_test = list(popqa_test)


# def load_jsonl_file(jsonl_path, max_samples=None):
#     """Loads .jsonl files line by line."""
#     with open(jsonl_path, 'r', encoding='utf-8') as f:
#         lines = f.readlines()
#     data = [json.loads(line) for line in lines]
#     return data if max_samples is None else data[:max_samples]

# def load_gsm_symbolic_data(base_dir, max_samples_per_file=200):
#     """Loads multiple JSONL files from gsm-symbolic_data."""
#     combined = []
#     for fname in ["GSM_p1.jsonl", "GSM_p2.jsonl", "GSM_symbolic.jsonl"]:
#         path = os.path.join(base_dir, fname)
#         if os.path.exists(path):
#             combined.extend(load_jsonl_file(path, max_samples_per_file))
#     return combined

# # --- Paths (adjust if needed) ---
# DATASET_ROOT = "../dataset"

# gsm_sym_data = load_gsm_symbolic_data(os.path.join(DATASET_ROOT, "gsm-symbolic_data"))
# human_eval_path = os.path.join(DATASET_ROOT, "HumanEval", "HumanEval.jsonl")
# human_eval_data = load_jsonl_file(human_eval_path, max_samples=600)

# print(f"Loaded {len(gsm_sym_data)} samples from GSM Symbolic")
# print(f"Loaded {len(human_eval_data)} samples from HumanEval")


# other_running_set_name_list = ['ceval_liberal', 'gsm8k', 'mgsm', 'popqa', 'gsm-symbolic', 'human_eval', 'human_eval', 'mbpp'] # mbpp,, hoppingtoolate， , 'mbpp', 'popqa',
# other_running_set_name_list = [ 'gsm-symbolic', 'human_eval', 'human_eval', 'mbpp']
other_running_set_name_list = ['ceval_liberal', 'gsm8k', 'mgsm', 'popqa'] # mbpp,, hoppingtoolate， , 'mbpp', 'popqa',


other_dataset = None

hs_cache_no_cot_other_all = {}

for other_running_set_name in other_running_set_name_list:
    
    if other_running_set_name == 'mbpp':
        other_dataset = mbpp_ds_full_test
        print(f"All {other_running_set_name} answeres are of type: {pd.DataFrame(other_dataset)[1].apply(type).unique()}")
    elif other_running_set_name == 'gsm8k':
        other_dataset = gsm8k_ds_main_test
        print(f"All {other_running_set_name} answeres are of type: {pd.DataFrame(other_dataset)[1].apply(type).unique()}")
    elif other_running_set_name == 'mgsm': #multilingual gsm8k
        other_dataset = mgsm_zh_test + mgsm_de_test + mgsm_bn_test + mgsm_ja_test + mgsm_te_test
        print(f"All {other_running_set_name} answeres are of type: {pd.DataFrame(other_dataset)[1].apply(type).unique()}")
    elif other_running_set_name == 'ceval_liberal':
        other_dataset = ceval_chi_test + ceval_his_test + ceval_pol_test + ceval_mar_test # + ceval_bus_test
        print(f"All {other_running_set_name} answeres are of type: {pd.DataFrame(other_dataset)[1].apply(type).unique()}")
    elif other_running_set_name == 'popqa': #multilingual gsm8k
        other_dataset = popqa_test
        print(f"All {other_running_set_name} answeres are of type: {pd.DataFrame(other_dataset)[1].apply(type).unique()}")
    elif other_running_set_name == 'gsm-symbolic':
        other_dataset = gsm_sym_data
        print(f"All {other_running_set_name} answeres are of type: {pd.DataFrame(other_dataset)[1].apply(type).unique()}")
    elif other_running_set_name == 'human_eval':
        other_dataset = human_eval_data
        print(f"All {other_running_set_name} answeres are of type: {pd.DataFrame(other_dataset)[1].apply(type).unique()}")
    elif other_running_set_name == 'gsm_symbolic':
        other_dataset = gsm_sym_data
        print(f"All {other_running_set_name} answeres are of type: {pd.DataFrame(other_dataset)[1].apply(type).unique()}")


    print(f'#####Running on {other_running_set_name} test set')
    print(f'The size is {len(other_dataset)}')

    layers_to_cache_other = list(range(model_layers_num))
    print('layers_to_cache_other: ',layers_to_cache_other)
    hs_cache_no_cot_other = {}
    queries_batch = []
    batch_size = 4

    for ix, entry in tqdm(enumerate(other_dataset)):

        if other_running_set_name == 'gsm8k':
            query = 'Q: ' + entry['question'] + "\nA: "
        elif other_running_set_name == 'mbpp':
            query = 'Q: ' + entry['text'] + "\nA: "
        elif other_running_set_name == 'mgsm':
            query = 'Q: ' + entry[0] + "\nA: "
        elif other_running_set_name == 'ceval_liberal':
            query = 'Q: ' + entry + "\nA: "
        elif other_running_set_name == 'popqa':
            query = 'Q: ' + entry['question'] + "\nA: "

        queries_batch.append(query)

        if len(queries_batch) == batch_size or ix == len(other_dataset) - 1:
            output = generate_outputs(queries_batch)

            for layer in layers_to_cache_other:
                if layer not in hs_cache_no_cot_other:
                    hs_cache_no_cot_other[layer] = output["hidden_states"][layer][: ,-1 , :].cpu() #bs * tok * dims
                else:
                    hs_cache_no_cot_other[layer] = torch.cat((hs_cache_no_cot_other[layer], output["hidden_states"][layer][: ,-1 , :].cpu()), dim=0)

            queries_batch = []
        torch.cuda.empty_cache()

    hs_cache_no_cot_other_all[other_running_set_name] = hs_cache_no_cot_other



[GSM8K Main Train] Trying to convert 'answer' to float...
[GSM8K Main Train] Sample converted answers:
0                      NOT_CONVERTIBLE
1                      NOT_CONVERTIBLE
2                      NOT_CONVERTIBLE
3                      NOT_CONVERTIBLE
4    3232666262121212699898133086208.0
Name: answer, dtype: object
[GSM8K Main Train] Unique types after conversion: [<class 'str'> <class 'float'>]
[GSM8K Main Train] Count of unique values: 1177
[GSM8K Main Train] ⚠️ 6295 answers could not be converted to float.

[MGSM_ZH] Trying to convert '1' to float...
[MGSM_ZH] Sample converted answers:
0       18.0
1        3.0
2    70000.0
3      540.0
4       20.0
Name: 1, dtype: float64
[MGSM_ZH] Unique types after conversion: [<class 'float'>]
[MGSM_ZH] Count of unique values: 134

[MGSM_DE] Trying to convert '1' to float...
[MGSM_DE] Sample converted answers:
0       18.0
1        3.0
2    70000.0
3      540.0
4       20.0
Name: 1, dtype: float64
[MGSM_DE] Unique types after conversio

KeyError: 1

In [ ]:
hs_cache_no_cot_other_all['mmlu-pro_600'] = hs_cache_no_cot
hs_cache_no_cot_other_all['mmlu-pro_3000'] = hs_cache_no_cot_3000


In [12]:
pd.DataFrame(other_dataset)#

,0
0,宋代在“十二经”的基础上加入了____，成了后人所说的“十三经”。
1,____是西域传来的乐器。
2,“让人不放心”属于____。
3,昆明话属于____。
4,语气词“呢”不能用在哪种疑问句中?____
...,...
741,历史人物的成功或失败，归根到底取决于____
742,生产力是人们____
743,唯物辩证法认为，发展的实质是____
744,社会基本矛盾运动的最终原因是____


# Let's take a look at the output vector shape

In [ ]:
for k, v in hs_cache_no_cot_other_all.items():
    print(f"{k} shape: {v[0].shape}")

# Let's take a look at the dataset

In [ ]:
# pd.DataFrame(gsm8k_ds_main['train']).head()
pd.DataFrame(gsm_sym_data).head()

In [ ]:
pd.DataFrame(other_dataset).head()

In [ ]:
list(hs_cache_no_cot_other_all.keys())

# save vectors

In [ ]:
save_path = '../reasoning_representations_outputs'
os.makedirs(save_path, exist_ok=True)  

torch.save(hs_cache_no_cot_other_all, os.path.join(save_path, f'{model_name}-base_hs_cache_no_cot_all.pt'))